In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from importlib import reload

import utils.models as models
import utils.plotting as plotting
import utils.dataloaders as dl
import utils.traintest as tt
import utils.adversarial as adv
import utils.eval as ev
import resnet
from tensorboardX import SummaryWriter



import params as hps

In [3]:
reload(hps)

<module 'params' from '/home/alexm/project/notebooks/gmm-robust/params.py'>

In [ ]:
device = torch.device('cuda:' + str(hps.gpu))
writer = SummaryWriter()

if hps.dataset=='MNIST':
    base_model = models.LeNetMadry().to(device)
    train_loader = dl.MNIST_train_loader
    noise_loader = dl.Noise_train_loader_MNIST
elif hps.dataset=='CIFAR10':
    base_model = resnet.ResNet50().to(device).to(device)
    train_loader = dl.CIFAR10_train_loader
    noise_loader = dl.Noise_train_loader_CIFAR10
    
noise_loader = dl.PrecomputeLoader(noise_loader)

In [4]:


if hps.use_gmm:
    loading_string = hps.dataset+'_n'+str(hps.n) 
    gmm = torch.load('SavedModels/gmm_'+loading_string+'.pth')
    model = models.RobustModel(base_model, gmm, hps.lam).to(device)
    model.loglam.requires_grad = False
else:
    model = base_model

saving_string = hps.dataset+'_lam'+str(hps.lam)+'_n'+str(hps.n)

lr = hps.lr

if hps.use_gmm:
    param_groups = [{'params':model.mm.parameters(),'lr':lr, 'weight_decay':0.},
                    {'params':model.base_model.parameters(),'lr':lr, 'weight_decay':hps.decay}]
else:
    param_groups = [{'params':model.parameters(),'lr':lr, 'weight_decay':hps.decay}]
    
optimizer = optim.Adam(param_groups)



In [5]:
for epoch in range(3):
    if epoch+1 in [50,75,90]:
        for group in optimizer.param_groups:
            group['lr'] *= .1
    tt.train_ACET(model, device, train_loader, noise_loader, optimizer, epoch, steps=hps.steps)
    correct, ave_conf = tt.test(model, device, dl.MNIST_test_loader)

Train Epoch: 0 [0/60000 (0%)]	Loss: 4.604506
Train Epoch: 0 [10000/60000 (17%)]	Loss: 4.468185
Train Epoch: 0 [20000/60000 (33%)]	Loss: 4.394171
Train Epoch: 0 [30000/60000 (50%)]	Loss: 4.420728
Train Epoch: 0 [40000/60000 (67%)]	Loss: 3.380988


KeyboardInterrupt: 

In [6]:
model.mm.alpha.sum()

tensor(0.3214, device='cuda:2', grad_fn=<SumBackward0>)

In [28]:
data = enumerate(dl.MNIST_test_loader).__next__()[1][0]
torch.logsumexp(model.mm(data.to(device).view(-1,784)),0).exp().mean()

tensor(0.1223, device='cuda:2', grad_fn=<MeanBackward1>)

In [29]:
data = enumerate(dl.EMNIST_test_loader).__next__()[1][0]
torch.logsumexp(model.mm(data.to(device).view(-1,784)),0).exp().mean()

tensor(0.0845, device='cuda:2', grad_fn=<MeanBackward1>)

In [30]:
data = enumerate(dl.FMNIST_test_loader).__next__()[1][0]
torch.logsumexp(model.mm(data.to(device).view(-1,784)),0).exp().mean()

tensor(0.0486, device='cuda:2', grad_fn=<MeanBackward1>)

In [27]:
model = torch.load('SavedModels/gmm_model_MNIST_lam-3.0_n1000.pth').to(device)

In [25]:
model.mm.alpha.sum()

tensor(0.3214, device='cuda:3', grad_fn=<SumBackward0>)

In [31]:
data = enumerate(dl.MNIST_test_loader).__next__()[1][0]

In [32]:
y = model(data.to(device))

In [39]:
df = pd.read_csv('results/gmm_model_jointMNIST_lam-3.0_n1000.csv')

'0.9558813'

In [46]:
df['MMC']

0    0.510050
1    0.164432
2    0.364298
3    0.101477
4    0.100063
5    0.100071
6    0.489142
Name: MMC, dtype: float64